<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Bounding_Boxes_For_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

vgg = VGG16(weights='imagenet',include_top=False,
            input_tensor=tf.keras.layers.Input(shape=(224,224,3)))

vgg.trainable = False
flatten = vgg.output
flatten = tf.keras.layers.Flatten()(flatten)

bbox_head = tf.keras.layers.Dense(128,activation='relu')(flatten)
bbox_head = tf.keras.layers.Dense(64,activation='relu')(bbox_head)
bbox_head = tf.keras.layers.Dense(32,activation='relu')(bbox_head)
bbox_head = tf.keras.layers.Dense(4,activation='sigmoid',name='bounding_box')(bbox_head)

softmax_head = tf.keras.layers.Dense(512,activation='relu')(flatten)
softmax_head = tf.keras.layers.Dropout(0.5)(softmax_head)
softmax_head = tf.keras.layers.Dense(512,activation='relu')(flatten)
softmax_head = tf.keras.layers.Dropout(0.5)(softmax_head)
softmax_head = tf.keras.layers.Dense(10,activation='softmax',name='class_head')(softmax_head)

model = tf.keras.models.Model(inputs=vgg.input,outputs=(bbox_head,softmax_head))

losses = {
    'bounding_box':'categorical_crossentropy',
    'class_head':'mean_squared_error'
}
loss_weights = {
    'bounding_box':1.0,
    'class_head':1.0
}
model.compile(optimizer='Adam',loss=losses,loss_weights=loss_weights,metrics=['accuracy'])
model.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
_____________________________